# Ouray Defensible Space - Create and Populate

Author: **Bryce A Young** (git bryceayoung) | 
Created: **2024-12-05** | 
Modified: **2025-02-15**

In this notebook, we detect fuel distributions within the defensible space of every building in Ouray County. We build the defensible space dataset here, which contains HIZ polygons, LiDAR metrics, number of adjacent structures, distance to nearest structure. 

Documents for cleaning and preparing raw data for the analysis are located in this repo under `workflows/data_prep`.

#### Data 
- (vector) Microsoft Building Footprints
- (raster) LiDAR-derived rasters

#### Workflow 
- Create HIZ boundaries around each building footprint
- Count number of homes within each HIZ
- Get zonal summary values of each LiDAR-derived raster for each HIZ

## Step 0: Setup Environment
---

In [2]:
import os
### Directory ###
# Repository
os.chdir(r'D:/_PROJECTS/P001_OurayParcel/ouray')
# Root workspace
ws = r'D:/_PROJECTS/P001_OurayParcel'

### Data paths ###
# Folder where all the data inputs and outputs will live
data = os.path.join(ws, 'data')
# Folder for geoms (microsoft building footprints, parcels, and county boundary)
geoms = os.path.join(data, 'county_geoms')
# Folder containing LiDAR-derived rasters
rasters = os.path.join(data, 'tiffs_from_las')
# Scratch folder
scratch = os.path.join(data, '_temp')
# Any final outputs go here
out = os.path.join(data, '_out')

# Ensure correct working directory
os.getcwd()

'D:\\_PROJECTS\\P001_OurayParcel\\ouray'

## Step 1: Create HIZ Boundaries around each Structure
---
**Background: obtaining building footprints**  
*I searched "Ouray County Colorado Microsoft Building Footprints" and found a Colorado state website that had parsed the Microsoft Building Footprints dataset into county datasets for all of Colorado. So I was able to directly download the Ouray footprints without having to do any data manipulation myself.*

*After downloading the footprints, I added them to ArcGIS Pro and viewed them on top of my LiDAR data. Especially the `zentropy` layer shows buildings very well. I noticed that the footprint geometries are offset from the location of the buildings in the LiDAR rasters. So I used the 'Transform' tool in ArcGIS Pro 3.4.0 to put control points and target points (links) with Rubbersheet (natural neighbor) as the transformation method. This operation brought the building footprints closer to the buildings as they appear in the Z-entropy LiDAR raster. This created 105 anchor points througout the county extent where buildings are present.*

*The resulting shapefile was saved to `buildings_rs_WKID26913.shp` where 'rs' denotes 'rubber sheet' transformation of the original footprint layer. Original footprints are saved in the same directory as `Ouray_County_Buildings.shp` and geopackage as `buildings_WKID26913.gpkg`.*

*In order to display my results, I randomized the building footprints and selected 9 random buildings which I plotted in an ArcGIS Pro layout with a 100m square buffer around the centroid. The maps in the layout show the pre-rubbersheet and post-rubbersheet building footprints on top of the Z-entropy LiDAR raster. This shows that the building footprints are well-aligned with the LiDAR rasters in order to ensure that the structures are not being considered in the LiDAR analysis.*

### Import geometries and run HIZ script
The function `simple_hiz` from `utils.HIZ` in this repo creates a single defensible space zone around each building footprint. Looking at the randomly selected buildings, 3m is a sufficient buffer. I will put this buffer in the parameters and say that the outer edge will be that buffer + 30m, so the outer edge will be 33m from the building footprint. 

30m is chosen because of the NFPA and Cohen definitions of defensible space that are supported by post fire assessments, such as the Lahaina post-fire investigation (Hedayati et al. 2024).

In [2]:
import geopandas as gpd

# Import building footprints
mbf = gpd.read_file(os.path.join(geoms, 'buildings_rs_WKID26913.shp'))
mbf.head()

,County,geometry
0,Ouray County,"POLYGON ((233065.876 4241049.368, 233062.927 4..."
1,Ouray County,"POLYGON ((242799.828 4243540.283, 242798.994 4..."
2,Ouray County,"POLYGON ((251221.424 4245433.145, 251222.638 4..."
3,Ouray County,"POLYGON ((244094.335 4241435.666, 244103.748 4..."
4,Ouray County,"POLYGON ((254081.763 4242776.159, 254083.688 4..."


In [3]:
from utils.HIZ import simple_hiz

# Create defensible space zone around all building footprints, saving as its own gdf
hiz = simple_hiz(mbf)
hiz.head() # Preview data

,geometry
0,"POLYGON ((233082.966 4241022.314, 233080.295 4..."
1,"POLYGON ((242805.059 4243571.853, 242808.129 4..."
2,"POLYGON ((251182.204 4245395.048, 251179.74 42..."
3,"POLYGON ((244062.393 4241433.743, 244062.359 4..."
4,"POLYGON ((254114.554 4242761.882, 254113.584 4..."


In [4]:
# Save to file
hiz.to_file(os.path.join(geoms, 'hiz_WKID26913.gpkg'), driver='GPKG')
print('hiz geom saved to file!')

hiz geom saved to file!


## Step 2: Structure Density
---

In order to obtain a proxy for structure density, we are going to count the number of adjacent structures in the vicinity of each home. We will also compute distance to nearest structure. We will append these counts to the HIZ geopackage.

In [5]:
# Count number of structures within defensible space
from utils.HIZ import structures_in_hiz

counts_df = structures_in_hiz(mbf, hiz)
counts_df.head()

,footprint_index,intersections
0,0,0
1,1,2
2,2,0
3,3,0
4,4,0


In [6]:
counts_df.shape

(4533, 2)

In [7]:
# Save df to csv in scratch folder
counts_df.to_csv(os.path.join(scratch, 'structure_count.csv'))

In [8]:
# Compute minimum structure separation distance per structure
from utils.HIZ import min_ssd

mbf = min_ssd(mbf)
mbf.head()

,County,geometry,min_ssd
0,Ouray County,"POLYGON ((233065.876 4241049.368, 233062.927 4...",221.938317
1,Ouray County,"POLYGON ((242799.828 4243540.283, 242798.994 4...",11.936200
2,Ouray County,"POLYGON ((251221.424 4245433.145, 251222.638 4...",213.672097
3,Ouray County,"POLYGON ((244094.335 4241435.666, 244103.748 4...",183.675975
4,Ouray County,"POLYGON ((254081.763 4242776.159, 254083.688 4...",101.386718


In [9]:
# Save mbf to file in scratch folder
mbf.to_file(os.path.join(scratch, 'mbf_wui_ssd.gpkg'), driver='GPKG')

## Step 3: Get canopy cover info for each HIZ
---

#### Read in LiDAR-derived rasters

I created these rasters in R using the **lidR** package and methods documented in this repository under `r_workflows`.

#### Get raster values per HIZ across the county

Recall the geodataframes of home ignition zones that we created at the beginning of this workflow. It contains geometries for **4533** building footprins and their home ignition zones. We're going to look at the vegetation within each HIZ to get an idea of the vegetation profile around each home.

For the purposes of reducing wildfire risk to individual homes, it's important that we analyze raster values within each buffer zone. GeoPandas comes in handy for this task.

Our workflow for this calculation - written in plain English - is as follows:
1. **Mask raster with geometries**: Recall that each home ignition zone gdf is a geoseries of Shapely geometries. We need to iterate through **each** of these **4533** objects and use them to mask our raster, object by object.
2. **Calculate average raster values within each HIZ**: once the mask is applied to only include a single building buffer, we'll take the average of all the pixels in the area. 
3. **Store respective canopy cover value per HIZ as a new column in the HIZ dataframe:** The average values will be stored as columns and each value will correspond to the geometry in the same row.

In [2]:
import geopandas as gpd
hiz = gpd.read_file(os.path.join(geoms, 'hiz_WKID26913.gpkg'))

In [3]:
hiz.head()

,geometry
0,"POLYGON ((233082.966 4241022.314, 233080.295 4..."
1,"POLYGON ((242805.059 4243571.853, 242808.129 4..."
2,"POLYGON ((251182.204 4245395.048, 251179.74 42..."
3,"POLYGON ((244062.393 4241433.743, 244062.359 4..."
4,"POLYGON ((254114.554 4242761.882, 254113.584 4..."


In [ ]:
from utils.HIZ import raster_stats

# File naming conventions, starting with aoi (a county, community, etc.) and ending with spatial reference
wkid = 'WKID26913'
aoi = 'ouray'

# Define names of rasters for dynamic column naming in gdf output
rnames = ['cc0_2m', 'cc2_4m', 'cc4_8m', 'cc8_40m']
raster_paths = [os.path.join(rasters, f'{aoi}_cc0_2m_{wkid}.tif'),
                os.path.join(rasters, f'{aoi}_cc2_4m_{wkid}.tif'),
                os.path.join(rasters, f'{aoi}_cc4_8m_{wkid}.tif'),
                os.path.join(rasters, f'{aoi}_cc8_40m_{wkid}.tif')
                ]

hiz_full = raster_stats(hiz, raster_paths, rnames, stats=['mean'])
hiz_full.head()

Error processing D:/_PROJECTS/P001_OurayParcel\data\tiffs_from_las\ouray_cc0_2m_WKID26913.tif for HIZ 0: Input shapes do not overlap raster.
Error processing D:/_PROJECTS/P001_OurayParcel\data\tiffs_from_las\ouray_cc0_2m_WKID26913.tif for HIZ 31: Input shapes do not overlap raster.
Error processing D:/_PROJECTS/P001_OurayParcel\data\tiffs_from_las\ouray_cc0_2m_WKID26913.tif for HIZ 196: Input shapes do not overlap raster.
Error processing D:/_PROJECTS/P001_OurayParcel\data\tiffs_from_las\ouray_cc0_2m_WKID26913.tif for HIZ 363: Input shapes do not overlap raster.
Error processing D:/_PROJECTS/P001_OurayParcel\data\tiffs_from_las\ouray_cc0_2m_WKID26913.tif for HIZ 573: Input shapes do not overlap raster.
Error processing D:/_PROJECTS/P001_OurayParcel\data\tiffs_from_las\ouray_cc0_2m_WKID26913.tif for HIZ 575: Input shapes do not overlap raster.
Error processing D:/_PROJECTS/P001_OurayParcel\data\tiffs_from_las\ouray_cc0_2m_WKID26913.tif for HIZ 817: Input shapes do not overlap raster.
Er

,geometry,mean_cc0_2m,mean_cc_2_4m,mean_cc4_8m,mean_cc8_40m
0,"POLYGON ((233082.966 4241022.314, 233080.295 4...",NaN,NaN,NaN,NaN
1,"POLYGON ((242805.059 4243571.853, 242808.129 4...",0.079294,0.119998,0.066202,0.003467
2,"POLYGON ((251182.204 4245395.048, 251179.74 42...",0.032280,0.038839,0.006272,0.000000
3,"POLYGON ((244062.393 4241433.743, 244062.359 4...",0.063696,0.097281,0.033223,0.082861
4,"POLYGON ((254114.554 4242761.882, 254113.584 4...",0.037428,0.219677,0.253634,0.000842


In [5]:
hiz_full.to_file(os.path.join(scratch, 'hiz_raster_WKID26913.gpkg'), driver='GPKG', index=False)
print('scratch gpkg saved.')

scratch gpkg saved.


#### Append values to HIZ geopackages
We have three items: HIZ geoms, MBF centroids, and counts_df. We're going to import all three of them and join them to MBF centroids. We're using the centroids rather than the geometries of the building footprints because I believe that will result in less ambiguity for the spatial join to the parcel geometries, which is required to append the tax assessor data and risk scores.

In [6]:
# Code to append values
import pandas as pd
import geopandas as gpd

df = pd.read_csv(os.path.join(scratch, 'structure_count.csv'))
mbf1 = gpd.read_file(os.path.join(scratch, 'mbf_centroid_wui_class.gpkg'))
mbf2 = gpd.read_file(os.path.join(scratch, 'mbf_wui_ssd.gpkg'))
hiz = gpd.read_file(os.path.join(scratch, 'hiz_raster_WKID26913.gpkg'))

df.head()

,Unnamed: 0,footprint_index,intersections
0,0,0,0
1,1,1,2
2,2,2,0
3,3,3,0
4,4,4,0


In [7]:
mbf1.head()

,County,wui_class,geometry
0,Ouray County,5,POINT (233065.909 4241052.6)
1,Ouray County,5,POINT (242785.257 4243534.402)
2,Ouray County,1,POINT (251212.196 4245423.292)
3,Ouray County,5,POINT (244097.801 4241432.894)
4,Ouray County,5,POINT (254078.896 4242776.879)


In [8]:
mbf2.head()

,County,min_ssd,geometry
0,Ouray County,221.938317,"POLYGON ((233065.876 4241049.368, 233062.927 4..."
1,Ouray County,11.936200,"POLYGON ((242799.828 4243540.283, 242798.994 4..."
2,Ouray County,213.672097,"POLYGON ((251221.424 4245433.145, 251222.638 4..."
3,Ouray County,183.675975,"POLYGON ((244094.335 4241435.666, 244103.748 4..."
4,Ouray County,101.386718,"POLYGON ((254081.763 4242776.159, 254083.688 4..."


In [9]:
hiz.head()

,mean_cc0_2m,mean_cc_2_4m,mean_cc4_8m,mean_cc8_40m,geometry
0,NaN,NaN,NaN,NaN,"POLYGON ((233082.966 4241022.314, 233080.295 4..."
1,0.079294,0.119998,0.066202,0.003467,"POLYGON ((242805.059 4243571.853, 242808.129 4..."
2,0.032280,0.038839,0.006272,0.000000,"POLYGON ((251182.204 4245395.048, 251179.74 42..."
3,0.063696,0.097281,0.033223,0.082861,"POLYGON ((244062.393 4241433.743, 244062.359 4..."
4,0.037428,0.219677,0.253634,0.000842,"POLYGON ((254114.554 4242761.882, 254113.584 4..."


Okay this is a bit of a mess and I'll want to figure out my workflow better in a script...next time. The ideal workflow would keep saving and deleting these scratch files. But for the first time through, I want to keep them because I can ensure better quality through a manual process like this.

In [ ]:
# Ensure identical number of observations
print(df.shape)
print(hiz.shape)
print(mbf1.shape)
print(mbf2.shape)

(4533, 3)
(4533, 5)
(4533, 3)
(4533, 3)


In [ ]:
# Prevent silent indexing errors
assert mbf1.index.equals(mbf2.index)
assert mbf1.index.equals(hiz.index)
assert mbf1.index.equals(df.index)

# Select columns to keep
mbf2_selected = mbf2[['min_ssd']]
hiz_selected = hiz[['mean_cc0_2m', 'mean_cc2_4m', 'mean_cc4_8m', 'mean_cc8_40m']]
df_selected = df[['intersections']]

# Join
mbf1 = mbf1.join([mbf2_selected, hiz_selected, df_selected])

mbf1.head()

,County,wui_class,geometry,min_ssd,mean_cc0_2m,mean_cc_2_4m,mean_cc4_8m,mean_cc8_40m,intersections
0,Ouray County,5,POINT (233065.909 4241052.6),221.938317,NaN,NaN,NaN,NaN,0
1,Ouray County,5,POINT (242785.257 4243534.402),11.936200,0.079294,0.119998,0.066202,0.003467,2
2,Ouray County,1,POINT (251212.196 4245423.292),213.672097,0.032280,0.038839,0.006272,0.000000,0
3,Ouray County,5,POINT (244097.801 4241432.894),183.675975,0.063696,0.097281,0.033223,0.082861,0
4,Ouray County,5,POINT (254078.896 4242776.879),101.386718,0.037428,0.219677,0.253634,0.000842,0


In [16]:
# Save updated GDF
mbf1.to_file(os.path.join(scratch, 'centr_wui_ssd_count_rast.gpkg'), driver='GPKG', index=False)
print('updated gdf saved.')

updated gdf saved.


In [ ]:
# Save final defensible space features to csv
mbf1.drop(columns=['geometry']).to_csv(os.path.join(out, 'hiz_feat.csv'), index=False)
print('csv saved to output folder.')

csv saved to output folder.


## Analysis of defensible space attributes
---
Next we are going to describe our dataset by looking at the distributions. This will be conducted under `workflows/analysis/hiz_analysis.ipynb`.